In [18]:
# From the book Machine Learning in Action

In [19]:
class TreeNode:
    def __init__(self, value, count, parent):
        self.value = value
        self.count = count
        self.node = None
        self.parent = parent
        self.children = {}

    def inc(self, count):
        self.count += count

    def disp(self, ind=1):
        print("  " * ind, self.value, " ", self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [20]:
# Create a root node
root_node = TreeNode("pyramid", 9, None)
root_node.children["eye"] = TreeNode("eye", 13, None)
root_node.disp()

   pyramid   9
     eye   13


In [21]:
root_node.children["phoenix"] = TreeNode("phoenix", 3, None)
root_node.disp()

   pyramid   9
     eye   13
     phoenix   3


In [22]:
def create_tree(dataset, min_support=1):
    header_table = {}
    # Iterate dataset twice
    for trans in dataset:
        for item in trans:
            header_table[item] = header_table.get(item, 0) + dataset[trans]

    # Remove item not meeting min support
    header_table_copy = header_table.copy()
    for k in header_table_copy.keys():
        if header_table[k] < min_support:
            del header_table[k]

    freq_itemset = set(header_table.keys())
    #     print('freq_itemset:', freq_itemset)
    if len(freq_itemset) == 0:
        return None, None

    for k in header_table:
        header_table[k] = [header_table[k], None]

    # Tree to be returned
    ret_tree = TreeNode("Null Set", 1, None)
    for trans_set, count in dataset.items():
        local = {}
        for item in trans_set:
            if item in freq_itemset:
                local[item] = header_table[item][0]
        if len(local) > 0:
            ordered_items = [
                v[0]
                for v in sorted(
                    local.items(),
                    key=lambda p: (
                        p[1],
                        p[0],
                    ),  # Sort by count first, then sort by alphabet
                    reverse=True,
                )
            ]
            update_tree(ordered_items, ret_tree, header_table, count)

    return ret_tree, header_table

In [23]:
def update_tree(items, in_tree, header_table, count):
    if items[0] in in_tree.children:
        in_tree.children[items[0]].inc(count)
    else:
        in_tree.children[items[0]] = TreeNode(items[0], count, in_tree)
        if header_table[items[0]][1] == None:
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1], in_tree.children[items[0]])
    if len(items) > 1:
        update_tree(items[1::], in_tree.children[items[0]], header_table, count)

In [24]:
def update_header(node_to_test, target_node):
    while node_to_test.node != None:
        node_to_test = node_to_test.node
    node_to_test.node = target_node

In [25]:
def load_simple_data():
    return [
        ["r", "z", "h", "j", "p"],
        ["z", "y", "x", "w", "v", "u", "t", "s"],
        ["z"],
        ["r", "x", "n", "o", "s"],
        ["y", "r", "x", "z", "q", "t", "p"],
        ["y", "z", "x", "e", "q", "s", "t", "m"],
    ]


#     return [['a', 'b', 'c'],
#     ['a', 'd', 'e'],
#     ['b', 'c', 'd'],
#     ['a', 'b', 'c', 'd'],
#     ['b', 'c'],
#     ['a', 'b', 'd'],
#     ['d', 'e'],
#     ['a', 'b', 'c', 'd'],
#     ['c', 'd', 'e'],
#     ['a', 'b', 'c']]

In [26]:
def create_initset(dataset):
    out = {}
    for trans in dataset:
        out[frozenset(trans)] = 1
    return out

In [27]:
initset = create_initset(load_simple_data())
initset

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [28]:
fp_tree, header = create_tree(initset, 3)
fp_tree.disp()
# header['e'][1].children['m'].value

   Null Set   1
     z   5
       r   1
       x   3
         y   3
           t   3
             s   2
             r   1
     x   1
       s   1
         r   1


In [29]:
def ascend_tree(leaf_node, prefix_path):
    """Ascends from leaf node to root"""
    if leaf_node.parent != None:
        prefix_path.append(leaf_node.value)
        ascend_tree(leaf_node.parent, prefix_path)

In [30]:
def find_prefix_path(base_path, tree_node):
    """Tree node comes from header table"""
    cond_paths = {}
    while tree_node != None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            cond_paths[frozenset(prefix_path[1:])] = tree_node.count
        tree_node = tree_node.node
    return cond_paths

In [31]:
# find_prefix_path('r', header['r'][1])

In [32]:
def mine_tree(in_tree, header_table, min_support, prefix, freq_item_list):
    big_list = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1][0])]

    for base_path in big_list:
        new_freq_set = prefix.copy()
        new_freq_set.add(base_path)
        freq_item_list.append(new_freq_set)
        cond_pattern_bases = find_prefix_path(base_path, header_table[base_path][1])
        my_cond_tree, my_head = create_tree(cond_pattern_bases, min_support)

        if my_head != None:
            print("conditional tree for: ", new_freq_set)
            my_cond_tree.disp(1)
            mine_tree(my_cond_tree, my_head, min_support, new_freq_set, freq_item_list)

In [33]:
freq_items = []
mine_tree(fp_tree, header, 2, set([]), freq_items)

conditional tree for:  {'r'}
   Null Set   1
     z   2
       x   1
     x   1
conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'x', 'y'}
   Null Set   1
     z   3
conditional tree for:  {'s'}
   Null Set   1
     x   3
       z   2
         y   2
           t   2
conditional tree for:  {'s', 'z'}
   Null Set   1
     x   2
conditional tree for:  {'s', 'y'}
   Null Set   1
     z   2
       x   2
conditional tree for:  {'s', 'x', 'y'}
   Null Set   1
     z   2
conditional tree for:  {'s', 't'}
   Null Set   1
     z   2
       y   2
         x   2
conditional tree for:  {'s', 'x', 't'}
   Null Set   1
     z   2
       y   2
conditional tree for:  {'s', 'x', 'y', 't'}
   Null Set   1
     z   2
conditional tree for:  {'s', 'y', 't'}
   Null Set   1
     z   2
conditional tree for:  {'t'}
   Null Set   1
     z   3
       y   3
         x   3
conditional tree for:  {'x', 't'}
   Null Set   1
     z   3
       y   3
conditional tree for:  {'

In [34]:
freq_items

[{'r'},
 {'r', 'z'},
 {'r', 'x'},
 {'y'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'y', 'z'},
 {'s'},
 {'s', 'z'},
 {'s', 'x', 'z'},
 {'s', 'y'},
 {'s', 'x', 'y'},
 {'s', 'x', 'y', 'z'},
 {'s', 'y', 'z'},
 {'s', 't'},
 {'s', 't', 'x'},
 {'s', 't', 'x', 'z'},
 {'s', 't', 'x', 'y'},
 {'s', 't', 'x', 'y', 'z'},
 {'s', 't', 'z'},
 {'s', 't', 'y'},
 {'s', 't', 'y', 'z'},
 {'s', 'x'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]